<a href="https://colab.research.google.com/github/ziadelsharawy/Test/blob/main/Neural_Network_Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy scikit-learn tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
df=pd.read_csv(r"/content/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [ ]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
for i in object_cols:
    print(df[i].value_counts())

Sex
M    725
F    193
Name: count, dtype: int64
ChestPainType
ASY    496
NAP    203
ATA    173
TA      46
Name: count, dtype: int64
RestingECG
Normal    552
LVH       188
ST        178
Name: count, dtype: int64
ExerciseAngina
N    547
Y    371
Name: count, dtype: int64
ST_Slope
Flat    460
Up      395
Down     63
Name: count, dtype: int64


In [ ]:
data_num=["Age","Oldpeak","RestingBP","Cholesterol","MaxHR",]
data_cat=["Sex","ChestPainType","FastingBS","RestingECG","ExerciseAngina","ST_Slope"]

In [ ]:
for feature in data_num:
    categories = df[feature].unique()
    samples = [df[df[feature] == category]['HeartDisease'] for category in categories]
    f_statistic, p_value = f_oneway(*samples)

    print(f"ANOVA for {feature}:")
    print("F-statistic:", f_statistic)
    print("P-value:", p_value)
    print()
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis - There is a significant relationship between the variables.")
else:
    print("Fail to reject the null hypothesis - There is no significant relationship between the variables.")

ANOVA for Age:
F-statistic: 2.801749993947974
P-value: 2.2105230499409783e-09

ANOVA for Oldpeak:
F-statistic: 5.577384468897429
P-value: 6.240955334007274e-29

ANOVA for RestingBP:
F-statistic: 1.4646570159040155
P-value: 0.011298150619295956

ANOVA for Cholesterol:
F-statistic: 1.7858449032462411
P-value: 1.2507667641948241e-08

ANOVA for MaxHR:
F-statistic: 2.414772385144937
P-value: 8.761081954678646e-13

Reject the null hypothesis - There is a significant relationship between the variables.


In [ ]:
for feature in data_cat:
    # Create a contingency table
    contingency_table = pd.crosstab(df[feature], df['HeartDisease'])

    # Perform the chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Print the results
    print(f"Chi-square Test for {feature}:")
    print("Chi-square statistic:", chi2)
    print("P-value:", p)
    print("Degrees of freedom:", dof)
    print("Expected frequencies:")
    print(expected)
    print()
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis - There is a significant relationship between the variables.")
else:
    print("Fail to reject the null hypothesis - There is no significant relationship between the variables.")

Chi-square Test for Sex:
Chi-square statistic: 84.14510134633775
P-value: 4.597617450809164e-20
Degrees of freedom: 1
Expected frequencies:
[[ 86.19825708 106.80174292]
 [323.80174292 401.19825708]]

Chi-square Test for ChestPainType:
Chi-square statistic: 268.06723902181767
P-value: 8.08372842808765e-58
Degrees of freedom: 3
Expected frequencies:
[[221.52505447 274.47494553]
 [ 77.26579521  95.73420479]
 [ 90.66448802 112.33551198]
 [ 20.54466231  25.45533769]]

Chi-square Test for FastingBS:
Chi-square statistic: 64.3206790933594
P-value: 1.0573018731809955e-15
Degrees of freedom: 1
Expected frequencies:
[[314.42265795 389.57734205]
 [ 95.57734205 118.42265795]]

Chi-square Test for RestingECG:
Chi-square statistic: 10.931469339140978
P-value: 0.0042292328167544925
Degrees of freedom: 2
Expected frequencies:
[[ 83.96514161 104.03485839]
 [246.53594771 305.46405229]
 [ 79.49891068  98.50108932]]

Chi-square Test for ExerciseAngina:
Chi-square statistic: 222.25938271530583
P-value: 2.9

In [ ]:
encoder = OneHotEncoder(sparse_output=False)


one_hot_encoded = encoder.fit_transform(df[object_cols])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(object_cols))


df_encoded = pd.concat([df, one_hot_df], axis=1)


df_encoded = df_encoded.drop(object_cols, axis=1)
df_encoded.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,49,160,180,0,156,1.0,1,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,37,130,283,0,98,0.0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,48,138,214,0,108,1.5,1,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,54,150,195,0,122,0.0,0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
df_encoded.count()

Age                  918
RestingBP            918
Cholesterol          918
FastingBS            918
MaxHR                918
Oldpeak              918
HeartDisease         918
Sex_F                918
Sex_M                918
ChestPainType_ASY    918
ChestPainType_ATA    918
ChestPainType_NAP    918
ChestPainType_TA     918
RestingECG_LVH       918
RestingECG_Normal    918
RestingECG_ST        918
ExerciseAngina_N     918
ExerciseAngina_Y     918
ST_Slope_Down        918
ST_Slope_Flat        918
ST_Slope_Up          918
dtype: int64

In [ ]:
X=df_encoded.drop(columns=["HeartDisease"])
y=df["HeartDisease"]

In [ ]:
y.nunique()

2

In [ ]:
model = Sequential()
model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/100
66/66 [==============================] - 4s 18ms/step - loss: 3.0683 - accuracy: 0.5303 - val_loss: 0.7011 - val_accuracy: 0.4865
Epoch 2/100
66/66 [==============================] - 1s 9ms/step - loss: 1.2886 - accuracy: 0.5409 - val_loss: 0.6655 - val_accuracy: 0.5000
Epoch 3/100
66/66 [==============================] - 0s 7ms/step - loss: 0.9297 - accuracy: 0.5576 - val_loss: 0.5719 - val_accuracy: 0.7162
Epoch 4/100
66/66 [==============================] - 0s 7ms/step - loss: 0.9010 - accuracy: 0.5742 - val_loss: 0.5739 - val_accuracy: 0.7838
Epoch 5/100
66/66 [==============================] - 1s 8ms/step - loss: 0.7374 - accuracy: 0.5939 - val_loss: 0.5568 - val_accuracy: 0.7162
Epoch 6/100
66/66 [==============================] - 1s 8ms/step - loss: 0.7043 - accuracy: 0.6121 - val_loss: 0.5745 - val_accuracy: 0.7162
Epoch 7/100
66/66 [==============================] - 1s 9ms/step - loss: 0.7379 - accuracy: 0.5742 - val_loss: 0.5700 - val_accuracy: 0.7162
Epoch 8/100


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')


6/6 [==============================] - 0s 3ms/step - loss: 0.4113 - accuracy: 0.8315
Test Accuracy: 83.15%
